# International Trade Analysis

Below is a project that analyzes international trade in the United States and Canada. The data for the analysis was obtained from the **Ampo\_Tech** database in SQL Server Management Studio. This initiative is crucial for economic plans, business choices, and policy measures within the international market. It offers a data-based framework for comprehending the flow of goods, services, and capital across international boundaries and their effects on national and global economies.

## Using SQL Server Management Studio to Create Database

The steps in creating the database and its associated schemas are guided by the techniques outlined in the [W3Schools](https:\www.w3schools.com\sql\sql_create_table.asp). The [W3Schools](https:\www.w3schools.com\sql\sql_foreignkey.asp) tutorial was used for all constraints in the Ampo\_Tech database for the project.

## Database Creation

```
CREATE DATABASE Ampo_Tech

```

- Create the following Schemas in the **Ampo\_Tech** database
    - dim
    - fact
    - stage

```
/*********************************************************/
/******************    Schema DDL       ******************/
/*********************************************************/

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'dim' ) 
BEGIN
    EXEC sp_executesql N'CREATE SCHEMA dim AUTHORIZATION dbo;'
END
;

GO

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'stg' ) 
BEGIN
    EXEC sp_executesql N'CREATE SCHEMA stg AUTHORIZATION dbo;'
END
;

GO

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'f' ) 
BEGIN
    EXEC sp_executesql N'CREATE SCHEMA f AUTHORIZATION dbo;'
END

```

## Dimension Creation

The project employed the [Ralph Kimball](https:\www.kimballgroup.com\data-warehouse-business-intelligence-resources\books\data-warehouse-dw-toolkit\) principles to create the following dimensions and facts for the project:

- dim.Products
- dim.Locations
- dim.Calendar
- f.Transaction\_fact

## Entity-Relationship Diagram (ERD) for Ampo\_Tech

The project employed [D](https:\app.diagrams.net\#G1VfG-P5W2IUFu50PnkcyGDwwhNFqVbReB#%7B%22pageId%22%3A%22tH9ZkHU0JGC2aQ4ivV1L%22%7D)rawio software to develop the relationships between the dimension tables and the fact table.   

![ERD](.\International_Trade.png)  

Creation dim.Products table

```
/*********************************************************/
/******************  Products DIM DDL   ******************/
/*********************************************************/

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Products')
BEGIN
CREATE TABLE dim.Products(
    product_id INT NOT NULL,
    product_name nvarchar(50) NULL,
    product_brand nvarchar(50) NULL
)


    ALTER TABLE dim.Products
    ADD CONSTRAINT PK_Products PRIMARY KEY(product_id);

    ALTER TABLE dim.Products
    ADD CONSTRAINT UC_Prods UNIQUE (ProductId);
END

GO

```

Creation dim.Locations table

```
/*********************************************************/
/******************  Locations DIM DDL   ******************/
/*********************************************************/

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Locations')
BEGIN
    CREATE TABLE dim.Locations(
    city_id int NOT NULL,
    store_city nvarchar(50) NOT NULL,
    store_country nvarchar(50) NOT NULL,
    sales_region nvarchar(50) NOT NULL,
    sales_district nvarchar(50) NOT NULL
)


ALTER TABLE dim.Locations
ADD CONSTRAINT PK_Locations_LUP PRIMARY KEY(pkLocations);

ALTER TABLE dim.Locations
ADD CONSTRAINT PK_Locations PRIMARY KEY (city_id);

END

GO

```

Creation dim.Calendar

```
/*********************************************************/
/****************** Calendar DIM Script ******************/
/*********************************************************/

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Calendar')
BEGIN
-- Create the Calendar table
CREATE TABLE dim.Calendar
(
    pkCalendar INT NOT NULL,
    DateValue DATE NOT NULL,
    Year INT NOT NULL,
    Quarter INT NOT NULL,
    Qtr VARCHAR(3) NOT NULL,
    Month INT NOT NULL,
    MonthName VARCHAR(10) NOT NULL,
    MonthShort VARCHAR(3) NOT NULL,
    Week INT NOT NULL,
    Day INT NOT NULL,
    DayName VARCHAR(10) NOT NULL,
    DayShort VARCHAR(3) NOT NULL,
    IsWeekday BIT NOT NULL,
    Weekday VARCHAR(3) NOT NULL
)


    ALTER TABLE dim.Calendar
    ADD CONSTRAINT PK_Calendar_Julian PRIMARY KEY(pkCalendar);

    ALTER TABLE dim.Calendar
    ADD CONSTRAINT UC_Calendar UNIQUE (DateValue);
END

GO

```

Creation fact table (f.Transation\_fact)

```
/*********************************************************/
/******************  Transaction_fact   ******************/
/*********************************************************/

IF EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'f' AND TABLE_NAME = 'Transaction_fact')
BEGIN 
    DROP TABLE f.Transaction_fact;
END

CREATE TABLE f.Transaction_fact (
    pkCalendar INT NULL,
    Transaction_Date DATE NULL,
    Transaction_Year INT NULL,
    Transaction_Month nvarchar(15) NULL,
    Product_id INT NULL,
    City_id INT NULL,
    Days_to_Ship INT NULL,
    Quantity SMALLINT NULL,
    Total_Sale MONEY NULL
)
 


ALTER TABLE f.Transaction_fact
ADD CONSTRAINT FK_Transaction_fact_Calendar FOREIGN KEY (pkCalendar)
    REFERENCES dim.Calendar(pkCalendar)

ALTER TABLE f.Transaction_fact CHECK CONSTRAINT FK_f.Transaction_fact_Calendar
GO

```

## Adding Constraints and Foreign Keys(FK) in the fact table

Possible constraints and foreign keys were added to the fact table to establish the relationship between the dimension tables and the fact table.

- FK\_Transaction\_fact\_Products
- FK\_Transaction\_fact\_Locations
- FK\_Transaction\_fact\_Calendar

### Creating Foreign Key on the fact table for the Products table

```
ALTER TABLE f.Transaction_fact
ADD  CONSTRAINT FK_Transaction_fact_products FOREIGN KEY(product_id)
REFERENCES dim.products(product_id)

```

### Creating Foreign Key on the fact table for the Locations table

```
ALTER TABLE f.Transaction_fact
ADD  CONSTRAINT FK_Transaction_fact_locations FOREIGN KEY(city_id)
REFERENCES dim.locations(city_id)

```

### Creating Foreign Key on the fact table for the Calendar table

```
ALTER TABLE f.Transaction_fact
ADD CONSTRAINT FK_Transaction_fact_Calendar FOREIGN KEY (pkCalendar)
REFERENCES dim.Calendar(pkCalendar)

```

## Importing Data from SQL\_TC\_I Database using SQL Server Management Studio (SSMS)

The project imported records from the SQL\_TC\_I database in SSMS since the database has a similar data structure as Ampo\_Tech

### Importing records from SQL\_TC\_I.dbo.products\_dim into Ampo\_Tech.dim.Products table in SSMS

```
INSERT INTO dim.Products (Product_ID, Product_Name, Product_Brand)
SELECT Product_ID,
       Product_Name,
       Product_Brand
FROM SQL_TC_I.dbo.products_dim

```

### Importing records from SQL\_TC\_I.dbo.cities\_dim into Ampo\_Tech.dim.Locations table in SSMS

```
INSERT INTO dim.Locations (City_ID, Store_City, Store_Country, Sales_District, Sales_Region)
SELECT City_ID,
       Store_City,
       Store_Country,
       Sales_District,
       Sales_Region
FROM SQL_TC_I.dbo.cities_dim

```

### Importing records from SQL\_TC\_I.dbo.transactions\_fact into Ampo\_Tech.f.Transaction\_fact table in SSMS

```
INSERT INTO f.Transaction_fact (Transaction_Date, Transaction_Year, Transaction_Month, Product_ID, City_ID, Days_to_Ship, Quantity, Total_Sale)
SELECT Transaction_Date,
       Transaction_Year,
       Transaction_Month,
       Product_ID,
       City_ID,
       Days_to_Ship,
       Quantity,
       Total_Sale
FROM SQL_TC_I.dbo.transactions_fact

```